In [1]:
%cd ../

import logging

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from dask.distributed import Client

import pandas as pd
import geopandas as gpd

from config.params import Params

from helper_fns import (
    read_forecasts_locally,
    read_observations_locally,
)

from hip.analysis.analyses.drought import (
    get_accumulation_periods,
)

from operational import run_full_index_pipeline

c:\Users\amine.barkaoui\OneDrive - World Food Programme\Documents\GitHub\anticipatory-action


c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\aa-env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
client = Client()

In [132]:
issue = 1

In [133]:
params = Params(iso='MOZ', issue=issue, index='SPI')

forecasts = read_forecasts_locally(
    f"data/{params.iso}/forecast/{params.iso}_SAB_tp_ecmwf_{str(params.issue).zfill(2)}/*.nc"
)
logging.info(f"Completed reading of forecasts for the whole {params.iso} country")

observations = read_observations_locally(f"data/{params.iso}/chirps")
logging.info(
    f"Completed reading of observations for the whole {params.iso} country"
)

In [5]:
triggers_df = pd.read_csv(f"data/{params.iso}/outputs/Plots/triggers.aa.python.spi.dryspell.2022.csv") 

if params.iso == 'MOZ':
    gdf = gpd.read_file(
        f"data/{params.iso}/shapefiles/moz_admbnda_2019_SHP/moz_admbnda_adm2_2019.shp"
    )
else:
    gdf = gpd.read_file(
        f"data/{params.iso}/shapefiles/global_adm2_wfpGE_UN_final/global_adm2_wfpGE_UN_final.shp"
    )
    gdf = gdf.loc[gdf.iso3 == params.iso]

In [139]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts, params.start_season, params.end_season, 2, params.max_index_period
)

In [140]:
accumulation_periods

{'JF': (1, 2),
 'FM': (2, 3),
 'MA': (3, 4),
 'AM': (4, 5),
 'JFM': (1, 2, 3),
 'FMA': (2, 3, 4),
 'MAM': (3, 4, 5)}

In [119]:
probs_merged_dataframes = [
    # TODO handle distributed computation
    run_full_index_pipeline(
        forecasts,
        observations,
        params,
        triggers_df,
        gdf,
        period_name,
        period_months,
    )
    for period_name, period_months in accumulation_periods.items()
]
logging.info(f"Completed analysis for the required indexes over MOZ country")

c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\hip-analysis-dev-env\lib\site-packages\xclim\sdba\utils.py:316: RuntimeWarning: All-NaN slice encountered in interp_on_quantiles
  warn(


In [120]:
probs_df, merged_df = zip(*probs_merged_dataframes)

probs_dashboard = pd.concat(probs_df)
probs_dashboard.to_csv(
    f"data/{params.iso}/outputs/Fbf_Pilot_MockUp/SPI_probabilities_season_{params.issue}.csv",
    index=False,
)

merged_dashboard = pd.concat(merged_df)
merged_dashboard.to_csv(
    f"data/{params.iso}/outputs/Fbf_Pilot_MockUp/SPI_probabilities_season_triggers_{params.issue}.csv",
    index=False,
)

In [121]:
results5 = merged_dashboard

In [10]:
results5dry = merged_dashboard

In [113]:
results6 = merged_dashboard

In [18]:
results6dry = merged_dashboard

In [105]:
results7 = merged_dashboard

In [26]:
results7dry = merged_dashboard

In [97]:
results8 = merged_dashboard

In [35]:
results8dry = merged_dashboard

In [89]:
results9 = merged_dashboard

In [43]:
results9dry = merged_dashboard

In [60]:
results10 = merged_dashboard

In [74]:
results10dry = merged_dashboard

In [123]:
pd.concat([results5, results6, results7, results8, results9, results10, results5dry, results6dry, results7dry, results8dry, results9dry, results10dry]).to_csv(f"AA/data/{params.iso}/outputs/Fbf_Pilot_MockUp/Python_probabilities_season_triggers.csv", index=False)

2023-10-13 17:58:45,180 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:64467', name: 2, status: running, memory: 0, processing: 0>
2023-10-13 17:58:45,427 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:64471', name: 1, status: running, memory: 0, processing: 0>
2023-10-13 17:58:45,482 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:64471'.
2023-10-13 17:58:45,490 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:64467'.
2023-10-14 06:12:28,293 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:64468', name: 3, status: running, memory: 0, processing: 0>
2023-10-14 06:12:28,309 - distributed.scheduler - WARNING - Received heartbeat from unregistered worker 'tcp://127.0.0.1:644